# Code to Preprocess Data 

In [1]:
import os
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy import fft # site says you should use fft not fftpack
# using rfft over fft is also faster as it only calculates the positive half of the freq domain (it's symmetric)
# checkout https://realpython.com/python-scipy-fft/ for more info, and explanation of what fft exactly is
# https://stackoverflow.com/questions/66675657/fast-fourier-transform-for-an-accelerometer-in-python also helped
from scipy.stats import entropy

In [2]:
#return sample rate - number of samples per second
def getSampleRate(data):
    seconds_elapsed = data['seconds_elapsed']
    sample_rates = []
    for s in seconds_elapsed:
        sample_rate = 0
        temp = int(s)
        for i in range(len(seconds_elapsed)):
            if int(seconds_elapsed[i]) == temp:
                sample_rate+=1
        sample_rates.append(sample_rate)
        
    return np.average(sample_rates)

In [3]:
def butter_lowpass(cutoff, nyq_freq, order=3):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = signal.butter(order, normal_cutoff, btype='lowpass')
    return b, a

#3rd order low pass butter worth filter
def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=3):
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [4]:
#build list of features for accelerometer data
def pre_process_acc_data(data):
    seconds_elapsed = data['seconds_elapsed'].to_numpy()
    sample_rate = 0
    for s in seconds_elapsed:
        if int(s) == 1:
            sample_rate+=1
    signal_length = int(data['seconds_elapsed'].iloc[-1]) * sample_rate
    data = data.drop(columns=['time', 'seconds_elapsed'])
    
    medDF = data.apply(signal.medfilt, kernel_size = 3)
    
    #Split into x,y,z after applying median filter
    med_x = medDF['x']
    med_y = medDF['y']
    med_z = medDF['z']

    #third order low-pass butterowrth filter
    cutoff_frequency = 20

    butterW_x = butter_lowpass_filter(med_x, cutoff_frequency, sample_rate/2)
    butterW_y = butter_lowpass_filter(med_y, cutoff_frequency, sample_rate/2)
    butterW_z = butter_lowpass_filter(med_z, cutoff_frequency, sample_rate/2)

    #extracting gravity from accelerometer
    cutoff_frequency = 0.3

    tGravityAcc_x = butter_lowpass_filter(butterW_x, cutoff_frequency, sample_rate/2)
    tGravityAcc_y = butter_lowpass_filter(butterW_y, cutoff_frequency, sample_rate/2)
    tGravityAcc_z = butter_lowpass_filter(butterW_z, cutoff_frequency, sample_rate/2)

    #extracting body from accelerometer
    tBodyAcc_x = np.subtract(butterW_x, tGravityAcc_x)
    tBodyAcc_y = np.subtract(butterW_y, tGravityAcc_y)
    tBodyAcc_z = np.subtract(butterW_z, tGravityAcc_z)

    #calculating magnitude
    tBodyAccMag = np.sqrt(np.square(tBodyAcc_x) + np.square(tBodyAcc_y) + np.square(tBodyAcc_z))
    tGravityAccMag = np.sqrt(np.square(tGravityAcc_x) + np.square(tGravityAcc_y) + np.square(tGravityAcc_z))


    #jerk is change of in acceleration in a unit time
    # old code
    # you're not taking the difference in acceleration
    # and you're not dividing by the time this change took place in
    # seconds_elapsed will continue to increase, it is not the difference
    #tBodyAccJerk_x = np.divide(tBodyAcc_x,seconds_elapsed)
    #tBodyAccJerk_y = np.divide(tBodyAcc_y,seconds_elapsed)
    #tBodyAccJerk_z = np.divide(tBodyAcc_z,seconds_elapsed)
    
    # if all 3 arrays are the same length then we can join these three loops into 1
    tBodyAccJerk_x = []
    for i in range(0, len(tBodyAcc_x)-1):
        tBodyAccJerk_x.append((tBodyAcc_x[i+1]-tBodyAcc_x[i])/(seconds_elapsed[i+1]-seconds_elapsed[i]))
    tBodyAccJerk_x.append(tBodyAccJerk_x[len(tBodyAccJerk_x)-1])
    
    
    tBodyAccJerk_y = []
    for i in range(0, len(tBodyAcc_y)-1):
        tBodyAccJerk_y.append((tBodyAcc_y[i+1]-tBodyAcc_y[i])/(seconds_elapsed[i+1]-seconds_elapsed[i]))
    tBodyAccJerk_y.append(tBodyAccJerk_y[len(tBodyAccJerk_y)-1])
 
    tBodyAccJerk_z = []
    for i in range(0, len(tBodyAcc_z)-1):
        tBodyAccJerk_z.append((tBodyAcc_z[i+1]-tBodyAcc_z[i])/(seconds_elapsed[i+1]-seconds_elapsed[i]))
    tBodyAccJerk_z.append(tBodyAccJerk_z[len(tBodyAccJerk_z)-1])
        
        
    tBodyAccJerkMag = np.sqrt(np.square(tBodyAccJerk_x) + np.square(tBodyAccJerk_y) + np.square(tBodyAccJerk_z))

    #FAST FOURIER TRANSFORM
    #fbodyAcc-XYZ /FbodyAccJerk/ fbodyAccJerkMag
    fBodyAcc_x = fft.fft(tBodyAcc_x)
    fBodyAcc_x= np.abs(fBodyAcc_x)
    #fBodyAcc_x = np.linspace(0, sample_rate, len(fBodyAcc_x)) if we want to plot these'll be the x axis
    # but we need to change their name
    
    fBodyAcc_y = fft.fft(np.array(tBodyAcc_y))
    fBodyAcc_y= np.abs(fBodyAcc_y)# this changes output from complex to real valued
    #fBodyAcc_y = np.linspace(0, sample_rate, len(fBodyAcc_y))
    
    fBodyAcc_z = fft.fft(np.array(tBodyAcc_z))
    fBodyAcc_z= np.abs(fBodyAcc_z)
    #fBodyAcc_z = np.linspace(0, sample_rate, len(fBodyAcc_z))

    fBodyAccMag = np.sqrt(np.square(fBodyAcc_x) + np.square(fBodyAcc_y) + np.square(fBodyAcc_z))

    fBodyAccJerk_x = fft.fft(tBodyAccJerk_x)
    fBodyAccJerk_x= np.abs(fBodyAccJerk_x)
    #fBodyAccJerk_x = np.linspace(0, sample_rate, len(fBodyAccJerk_x))
    
    fBodyAccJerk_y = fft.fft(tBodyAccJerk_y)
    fBodyAccJerk_y= np.abs(fBodyAccJerk_y)
    #fBodyAccJerk_y = np.linspace(0, sample_rate, len(fBodyAccJerk_y))

    fBodyAccJerk_z = fft.fft(tBodyAccJerk_z)
    fBodyAccJerk_z= np.abs(fBodyAccJerk_z)
    #fBodyAccJerk_z = np.linspace(0, sample_rate, len(fBodyAccJerk_z))
    
    fBodyAccJerkMag = np.sqrt(np.square(fBodyAccJerk_x) + np.square(fBodyAccJerk_y) + np.square(fBodyAccJerk_z))
    
    features = {

        "tBodyAcc_x": tBodyAcc_x,"tBodyAcc_y": tBodyAcc_y,"tBodyAcc_z": tBodyAcc_z,
        "tGravityAcc_x": tGravityAcc_x,"tGravityAcc_y": tGravityAcc_y,"tGravityAcc_z": tGravityAcc_z,
        "tBodyAccJerk_x": tBodyAccJerk_x,"tBodyAccJerk_y": tBodyAccJerk_y,"tBodyAccJerk_z": tBodyAccJerk_z,
        
        "tBodyAccMag": tBodyAccMag,
        "tGravityAccMag": tGravityAccMag,
        "tBodyAccJerkMag": tBodyAccJerkMag,
        
        "fBodyAcc_x": fBodyAcc_x,"fBodyAcc_y": fBodyAcc_y,"fBodyAcc_z": fBodyAcc_z,
        #"tBodyAcc_x": tBodyAcc_x,"tBodyAcc_y": tBodyAcc_y,"tBodyAcc_z": tBodyAcc_z,
        
        #"tBodyAccJerkMag":tBodyAccJerkMag,
        "fBodyAccJerkMag":fBodyAccJerkMag,
        "fBodyAccMag":tBodyAccJerkMag,
        
        "fBodyAccJerk_x": fBodyAccJerk_x,"fBodyAccJerk_y": fBodyAccJerk_y,"fBodyAccJerk_z": fBodyAccJerk_z
    }
    return features

In [5]:
#build list of features for gyroscope data
def pre_process_gyro_data(data):
    seconds_elapsed = data['seconds_elapsed'].to_numpy()
    sample_rate = 0
    for s in seconds_elapsed:
        if int(s) == 1:
            sample_rate+=1
    signal_length = int(data['seconds_elapsed'].iloc[-1]) * sample_rate
    data = data.drop(columns=['time', 'seconds_elapsed'])
    
    medDF = data.apply(signal.medfilt, kernel_size = 3)
    
    #Split into x,y,z after applying median filter
    med_x = medDF['x']
    med_y = medDF['y']
    med_z = medDF['z']

    #third order low-pass butterowrth filter
    cutoff_frequency = 20

    butterW_x = butter_lowpass_filter(med_x, cutoff_frequency, sample_rate/2)
    butterW_y = butter_lowpass_filter(med_y, cutoff_frequency, sample_rate/2)
    butterW_z = butter_lowpass_filter(med_z, cutoff_frequency, sample_rate/2)

    #extracting gravity from gyro data
    cutoff_frequency = 0.3

    tGravityGyro_x = butter_lowpass_filter(butterW_x, cutoff_frequency, sample_rate/2)
    tGravityGyro_y = butter_lowpass_filter(butterW_y, cutoff_frequency, sample_rate/2)
    tGravityGyro_z = butter_lowpass_filter(butterW_z, cutoff_frequency, sample_rate/2)

    #extracting body from gyro data
    tBodyGyro_x = np.subtract(butterW_x, tGravityGyro_x)
    tBodyGyro_y = np.subtract(butterW_y, tGravityGyro_y)
    tBodyGyro_z = np.subtract(butterW_z, tGravityGyro_z)

    #calculating magnitude
    tBodyGyroMag = np.sqrt(np.square(tBodyGyro_x) + np.square(tBodyGyro_y) + np.square(tBodyGyro_z))
    tGravityGyroMag = np.sqrt(np.square(tGravityGyro_x) + np.square(tGravityGyro_y) + np.square(tGravityGyro_z))


    #jerk is change of gyro data in unit time
    #old code
    #tBodyGyroJerk_x = np.divide(tBodyGyro_x,seconds_elapsed)
    #tBodyGyroJerk_y = np.divide(tBodyGyro_y,seconds_elapsed)
    #tBodyGyroJerk_z = np.divide(tBodyGyro_z,seconds_elapsed)
    
    # if all 3 arrays are the same length then we can join these three loops into 1
    tBodyGyroJerk_x = []
    for i in range(0, len(tBodyGyro_x)-1):
        tBodyGyroJerk_x.append((tBodyGyro_x[i+1]-tBodyGyro_x[i])/(seconds_elapsed[i+1]-seconds_elapsed[i]))
    tBodyGyroJerk_x.append(tBodyGyroJerk_x[len(tBodyGyroJerk_x)-1])
        
    tBodyGyroJerk_y = []
    for i in range(0, len(tBodyGyro_y)-1):
        tBodyGyroJerk_y.append((tBodyGyro_y[i+1]-tBodyGyro_y[i])/(seconds_elapsed[i+1]-seconds_elapsed[i]))
    tBodyGyroJerk_y.append(tBodyGyroJerk_y[len(tBodyGyroJerk_y)-1])
        
        
    tBodyGyroJerk_z = []
    for i in range(0, len(tBodyGyro_z)-1):
        tBodyGyroJerk_z.append((tBodyGyro_z[i+1]-tBodyGyro_z[i])/(seconds_elapsed[i+1]-seconds_elapsed[i]))
    tBodyGyroJerk_z.append(tBodyGyroJerk_z[len(tBodyGyroJerk_z)-1])
    
    tBodyGyroJerkMag = np.sqrt(np.square(tBodyGyroJerk_x) + np.square(tBodyGyroJerk_y) + np.square(tBodyGyroJerk_z))

    #FAST FOURIER TRANSFORM
    #fbodyGyro-XYZ /FbodyGyroJerk/ fbodyGyroJerkMag
    fBodyGyro_x = fft.fft(tBodyGyro_x)
    fBodyGyro_x= np.abs(fBodyGyro_x)
    #fBodyGyro_x = np.linspace(0, sample_rate, len(fBodyGyro_x)) same as for accelerometer above
    
    fBodyGyro_y = fft.fft(tBodyGyro_y)
    fBodyGyro_y= np.abs(fBodyGyro_y)
    #fBodyGyro_y = np.linspace(0, sample_rate, len(fBodyGyro_y))
    
    fBodyGyro_z = fft.fft(tBodyGyro_z)
    fBodyGyro_z= np.abs(fBodyGyro_z)
    #fBodyGyro_z = np.linspace(0, sample_rate, len(fBodyGyro_z))

    fBodyGyroMag = np.sqrt(np.square(fBodyGyro_x) + np.square(fBodyGyro_y) + np.square(fBodyGyro_z))

    fBodyGyroJerk_x = fft.fft(tBodyGyroJerk_x)
    fBodyGyroJerk_x= np.abs(fBodyGyroJerk_x)
    #fBodyGyroJerk_x = np.linspace(0, sample_rate, len(fBodyGyroJerk_x))
    
    fBodyGyroJerk_y = fft.fft(tBodyGyroJerk_y)
    fBodyGyroJerk_y= np.abs(fBodyGyroJerk_y)
    #fBodyGyroJerk_y = np.linspace(0, sample_rate, len(fBodyGyroJerk_y))

    fBodyGyroJerk_z = fft.fft(tBodyGyroJerk_z)
    fBodyGyroJerk_z= np.abs(fBodyGyroJerk_z)
    #fBodyGyroJerk_z = np.linspace(0, sample_rate, len(fBodyGyroJerk_z))
    
    fBodyGyroJerkMag = np.sqrt(np.square(fBodyGyroJerk_x) + np.square(fBodyGyroJerk_y) + np.square(fBodyGyroJerk_z))
    
    features = {
        "tBodyGyro_x": tBodyGyro_x,"tBodyGyro_y": tBodyGyro_y,"tBodyGyro_z": tBodyGyro_z,
        "tGravityGyro_x": tGravityGyro_x,"tGravityGyro_y": tGravityGyro_y,"tGravityGyro_z": tGravityGyro_z,
        "tBodyGyroJerk_x": tBodyGyroJerk_x,"tBodyGyroJerk_y": tBodyGyroJerk_y,"tBodyGyroJerk_z": tBodyGyroJerk_z,
        
        "tBodyGyroMag": tBodyGyroMag,
        "tGravityGyroMag": tGravityGyroMag,
        "tBodyGyroJerkMag": tBodyGyroJerkMag,
        
        "fBodyGyro_x": fBodyGyro_x,"fBodyGyro_y": fBodyGyro_y,"fBodyGyro_z": fBodyGyro_z,
        #"tBodyGyro_x": tBodyGyro_x,"tBodyGyro_y": tBodyGyro_y,"tBodyGyro_z": tBodyGyro_z,
        
        #"tBodyGyroJerkMag":tBodyGyroJerkMag,
        "fBodyGyroJerkMag":fBodyGyroJerkMag,
        "fBodyGyroMag":tBodyGyroJerkMag
    }
    return features

### Set of variables to be estimated from the list of features
- mean():     Mean value
- std():      Standard deviation
- mad():      Median absolute deviation 
- max():      Largest value in array
- min():      Smallest value in array
- iqr():      Interquartile range 
- entropy():  Signal entropy
- skewness(): skewness of the frequency domain signal 
- kurtosis(): kurtosis of the frequency domain signal 

In [6]:
#return activity name and environment number 
#(subject is excluded as it is already known which subject performed which activity)

#example file name: Data Collection\Jogging\Jogging_1.7_(1)  
def get_details(filename):
    if 'Walking_Upstairs' in filename:
        return get_details_updown(filename)
    if 'Walking_Downstairs' in filename:
        return get_details_updown(filename)
    details = filename.split('\\')
    details = details[2]                 #Jogging_1.7_(1)
    details = details.split('_')
    activity = details[0]                #Jogging
    env = details[1].split('.')[1]       #7
    return activity, env

In [7]:
#this method is intended to have the same function as `get_details()` but is applied only for Walking UpDown Stairs activity

#example file name: Data Collection\Laying and Walking Up/Down Stairs\Walking_Upstairs_3.7.5_(1)
def get_details_updown(filename):
    details = filename.split('\\')
    details = details[2]                          #Walking_Upstairs_3.7.5_(1)
    details = details.split('_')
#    activity = details[0]+ " " + details[1]       #Walking_Upstairs
    activity = details[0]+ " Up/Down Stairs" 
    
    
    temp_env = details[2].split('.')
    if len(temp_env) == 3:
        env = temp_env[1] + '.' + temp_env[2]     #7.5
    else: 
        env = temp_env[1]                         #or 7 if there is only (..._3.7_(1)
    return activity, env

In [8]:
def dict_add(obj,key,value):
    obj[key]=value
    return obj

`def read_acc_data(listOfAcc):
    for f in listOfAcc:
        data = pd.read_csv(f) 
        features = pre_process_acc_data(data)
    return features`   

`def read_gyro_data(listOfGyro):
    for f in listOfGyro:
        data = pd.read_csv(f) 
        features = pre_process_gyro_data(data)`

In [9]:
#getting list of filenames 
listOfFiles = []
for (root,dirs,files) in os.walk('Data Collection'):
    for name in files: #for each file found
        if name != 'Metadata.csv': 
        #metadata includes the details of the smartphone used to record data - 
        #this is not needed
            listOfFiles.append(os.path.join(root, name))        

In [10]:
#Returns a list of Column names with features + the estimated variables (std, mean, max ...)
def get_col_names(df,string):
    listOfCols_final = []
    listOfCols = df.columns.tolist()

    for i in range(len(listOfCols)):
        listOfCols_final.append(listOfCols[i] + "_" + string)
        
    return listOfCols_final

In [11]:
def get_iqr_val(lower,upper):
    return np.subtract(upper,lower)

In [12]:
def get_entropy_val(features):
    entropy_values = []
    for key in features:
        #calculate entropy for each element in features
        pd_series = pd.Series(features[key])
        counts = pd_series.value_counts()
        value = entropy(counts)

        entropy_values.append(value)
    return entropy_values

#.astype('float64')

In [13]:
#to extract features for skweness and kurtosis
def prepare_features(features):
    new_features = {}
    for key in features:
        if 'fBodyAccMag' in key or 'fBodyAccJerk_' in key or 'fBodyGyro_' in key or 'fBodyAcc_' in key:
            dict_add(new_features,key,features[key])
            
    return new_features

__Skewness and kurtosis are only estimated on the following variables:__
- fbodyaccmag
- fbodygyroxyz
- fbodyaccjerkxyz
- fbodyaccxyz

In [14]:
final_listOfDicts = []


#returns a list of dictionaries with all the values - final
def read_all_files(listOfFiles):
    #list of files includes both gyro and acc data
    for f in listOfFiles:
        activity,env = get_details(f)
        #set title as file name
        nameOfFile = f
        data = pd.read_csv(f) 
        if 'Accelerometer.csv' in f: 
            features = pre_process_acc_data(data)
            # if accelerometer: start new row
            listOfCols_final = []
            result = []
            listOfCols_final.append('Activity')
            listOfCols_final.append('Environment')
            result.append(activity)
            result.append(env)
            final = {}
        if 'Gyroscope.csv' in f: 
            features = pre_process_gyro_data(data)
        
        feautures_df = pd.DataFrame.from_dict(features)
       
        #for median,max,min std
        timeDomainMetrics = feautures_df.describe()
        for num in [1,2,3,7]:
            result.extend(timeDomainMetrics.iloc[num].tolist())
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"mean"))
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"std"))
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"min"))
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"max"))
        
        
        #for iqr
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"iqr"))       
        result.extend(get_iqr_val(timeDomainMetrics.iloc[4].tolist(),timeDomainMetrics.iloc[6].tolist()))
    
    
        #for mean abs dev
        mad_vals = feautures_df.mad()
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"mad"))  
        result.extend(mad_vals)
        
        
        #for entropy - lack of order or predictability; gradual decline into disorder
        ent_vals = get_entropy_val(features)
        listOfCols_final.extend(get_col_names(timeDomainMetrics,"entropy"))
        result.extend(ent_vals)
        
        
        f_features = prepare_features(features) #features used for skewness and kurtosis
        f_features_df = pd.DataFrame.from_dict(f_features)
        
        #for skewness
        skew_vals = f_features_df.skew()
        listOfCols_final.extend(get_col_names(f_features_df,"skew"))
        result.extend(skew_vals)
        
        
        #for kurtosis
        kurt_vals = f_features_df.kurtosis()
        listOfCols_final.extend(get_col_names(f_features_df,"kurt"))
        result.extend(kurt_vals)

        #print(len(listOfCols_final))
        if 'Gyroscope.csv' in f:      
            for i in range(len(listOfCols_final)):
                dict_add(final,listOfCols_final[i],result[i])
            final_listOfDicts.append(final)
            
            
    return final_listOfDicts

In [15]:
final_dict = read_all_files(listOfFiles)

In [16]:
df = pd.DataFrame(final_dict) 
df.shape

(375, 281)

In [17]:
train=df.sample(frac=0.8,random_state=250) #random state is a seed value
test=df.drop(train.index)

In [18]:
train

,Activity,Environment,tBodyAcc_x_mean,tBodyAcc_y_mean,tBodyAcc_z_mean,tGravityAcc_x_mean,tGravityAcc_y_mean,tGravityAcc_z_mean,tBodyAccJerk_x_mean,tBodyAccJerk_y_mean,...,fBodyGyro_y_entropy,fBodyGyro_z_entropy,fBodyGyroJerkMag_entropy,fBodyGyroMag_entropy,fBodyGyro_x_skew,fBodyGyro_y_skew,fBodyGyro_z_skew,fBodyGyro_x_kurt,fBodyGyro_y_kurt,fBodyGyro_z_kurt
232,Walking Up/Down Stairs,6.4,-0.003247,-0.023978,0.010634,-0.024484,0.179190,-0.081896,0.004629,0.012094,...,8.157283,8.157283,8.157283,8.850032,19.068029,26.896403,23.455451,443.170211,981.912007,694.778330
150,Walking Up/Down Stairs,6.4,-0.003406,-0.017839,-0.001046,0.019053,0.165759,-0.114990,0.017374,-0.024135,...,7.892337,7.892337,7.892337,8.584967,25.019693,16.702488,20.045304,854.299906,457.956971,574.135297
24,Falling,25,0.006947,0.003593,-0.025026,-0.598838,-0.501942,-0.387593,-0.263061,-0.065136,...,6.873005,6.873005,6.873005,7.565075,4.983377,6.408874,6.201328,28.101364,50.730299,42.572040
197,Walking Up/Down Stairs,9.5,0.024677,-0.012084,0.005114,0.055345,0.142871,-0.089958,0.071191,-0.052932,...,8.000298,8.000298,8.000298,8.693097,30.456696,17.711650,24.487421,1189.201047,467.153788,810.408642
45,Jogging,10,-0.025575,0.010588,0.109328,1.266131,2.914257,-3.793095,-0.028779,-0.063142,...,8.036200,8.036200,8.036200,8.729011,10.524501,15.517435,9.807294,148.584406,409.558375,123.857022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Laying,13,0.000110,-0.000386,0.000497,0.056488,0.012439,-0.083881,-0.009123,-0.003118,...,8.101631,8.101631,8.101631,8.794463,6.546909,5.675385,9.142245,49.458135,35.174480,105.722129
252,Walking Up/Down Stairs,7.4,0.012513,-0.002455,0.005196,0.057085,0.182446,-0.146083,0.065104,-0.147402,...,8.064274,8.064274,8.064274,8.757094,20.095172,17.008845,26.262620,524.885492,399.225871,845.249341
208,Walking Up/Down Stairs,9,0.019292,-0.014177,0.020037,0.113986,0.179275,-0.046772,0.011292,0.037902,...,7.959572,7.959572,7.959572,8.652356,38.875812,20.533755,29.786797,1768.120008,635.361762,1157.887044
12,Falling,23,-0.006026,0.009920,0.013106,-0.455038,-0.715126,-0.111975,0.030000,0.039608,...,7.028952,7.028952,7.028952,7.721178,5.303515,6.782456,6.994913,31.853616,55.712232,55.142558


In [19]:
train.to_csv('Processed Data/train.csv',index=False)
test.to_csv('Processed Data/test.csv',index=False)